In [1]:
import sys, win32com.client
import pandas as pd
import numpy as np
import sapscript as ss
import re

    I_AGENT     Номер дебитора
    I_WERKS     Таблица заводов для National Contracts
    I_ADDTI     Попадание заказа на сборку в Кокпит (дата и время)
    I_VDATU_LOC Дата доставки из заказа, локальная
    I_VTIME_LOC Время доставки из заказа, локальное
    I_SHTAG_DT  Плановая дата отгрузки агенту. 
    E_DDLTI     Собрать и разместить не позднее (дата и время)
    E_DDLTI_BP  Собрать к без учёта паллетирования (отгрузки агенту)
    E_PALTI     Время начала палетирования
    E_DATES     Хранить собранный заказ ИМ по (включительно)
    CS_NORMS    Кокпит: нормативы сборки

In [33]:
def start_se24(sc,session):
    title = session.findById("wnd[0]").text
    if title != 'Тест метода GET_DDLTI: ведение параметров ввода':
        sc.run('se24')
        session.findById("wnd[0]/usr/ctxtSEOCLASS-CLSNAME").text = "zimk_getpar"
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/tbar[1]/btn[8]").press()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        methods = session.findById("wnd[0]/usr")
        
        for line in methods.children:
            if line.text == 'GET_DDLTI':
                top_index = line.CharTop
                break
        session.findById("wnd[0]/usr/lbl[45,"+str(top_index)+"]").setFocus()
        session.findById("wnd[0]/usr/lbl[45,"+str(top_index)+"]").caretPosition = 0
        session.findById("wnd[0]").sendVKey(2)
    return

def fill_fields(session, import_data):
    session.findById("wnd[0]/usr/txt[48,10]").text = import_data['I_VKORG']
    session.findById("wnd[0]/usr/txt[48,11]").text = import_data['I_VSBED']
    session.findById("wnd[0]/usr/txt[48,12]").text = import_data['I_AGENT']
    session.findById("wnd[0]/usr/txt[48,13]").text = import_data['I_WERKS']
    session.findById("wnd[0]/usr/txt[48,14]").text = import_data['I_ADDTI']
    session.findById("wnd[0]/usr/txt[48,15]").text = import_data['I_VDATU_LOC']
    session.findById("wnd[0]/usr/txt[48,16]").text = import_data['I_VTIME_LOC']
    session.findById("wnd[0]/usr/txt[48,17]").text = import_data['I_SHTAG_DT']
    return

def get_ddlti(session):
    ddlti = session.findById("wnd[0]/usr/lbl[48,23]").text
    ddlti0 = session.findById("wnd[0]/usr/lbl[48,24]").text
    params = session.findById("wnd[0]/usr/lbl[51,30]").text
    return ddlti0, ddlti, params

def test_print(row, ddlti0, ddlti, format_string, params):
    if row['E_DDLTI']==ddlti:
        delta = "\033[1;30;42mok     \033[0;0m"
    else:
        delta = "\033[1;30;41mError  \033[0;0m"
        
    print(format_string.format(row['I_VSBED'],
                               row['I_AGENT'],
                               row['I_WERKS'],
                               row['I_ADDTI'],
                               row['I_VDATU_LOC'],
                               row['I_VTIME_LOC'],
                               row['I_SHTAG_DT'],
                               ddlti0,
                               ddlti,
                               row['E_DDLTI'],
                               delta),
                               (*params.split()))
    return

def main(import_file, system_id, sheet_name='ND'):
    try:
        sc = ss.sap()
        session = sc.session()
        sc.check_system(system_id)
        #Проверяем, не запущена ли se24 и нужный метод. Если нет - запускаем
        start_se24(sc,session)
        
        #Читаем входные данные
        #import_data = pd.read_csv('get_ddlti_import.csv',dtype=str, delimiter='\t')
        import_data = pd.read_excel(import_file,dtype=str, skiprows=1, sheet_name=sheet_name)
        import_data = import_data.replace('nan','')
        format_string = '{:4s}{:11s}{:6s}{:21s}{:12s}{:10s}{:12s}{:19s}{:19s}{:19s}{:7s}'
        headers = ['Тип','Агент','Завод','ВремяВкокпит','ДатаДост','ВремяДост','ДатаОтгр','СобратьКбезАгента','СобратьК','Эталон','Check']
        
        #К этому моменту должен быть открыт метод ZIMK_GETPAR->GET_DDLTI() на ввод данных
        current_plant = ''
        for index, row in import_data.iterrows():
            fill_fields(session, row)
            session.findById("wnd[0]").sendVKey(8)
            ddlti0, ddlti, params = get_ddlti(session)
            session.findById("wnd[0]").sendVKey(3)
            if row['I_WERKS'] != current_plant:
                print('\r\nЗавод', row['I_WERKS'])
                print(format_string.format(*headers),'Params')
                current_plant = row['I_WERKS']
            test_print(row, ddlti0, ddlti, format_string, params)
    except:
        sc.error_print()
        
    finally:
        session = None

In [55]:
import_file = 'get_ddlti_import.xlsx'
system_id = 'ERD'
main(import_file, system_id, sheet_name='ED') #ND, ED или Regress


Завод 3426
Тип Агент      Завод ВремяВкокпит         ДатаДост    ВремяДост ДатаОтгр    СобратьКбезАгента  СобратьК           Эталон             Check   Params
IN  100004360  3426  30.05.19 05:00:00    05.11.2019                                           30.05.19 06:00:00  30.05.19 06:00:00  ok      60 60 0
IN  100004360  3426  30.05.19 05:00:01    05.11.2019                                           30.05.19 06:00:01  30.05.19 06:00:01  ok      60 60 0
IN  100004360  3426  30.05.19 04:00:59    05.11.2019                                           30.05.19 07:00:00  30.05.19 07:00:00  ok      60 60 0
IN  100004360  3426  30.05.19 14:36:00    05.11.2019                                           30.05.19 15:36:00  30.05.19 15:36:00  ok      60 60 0
IN  100004360  3426  30.05.19 17:00:00    05.11.2019                                           31.05.19 07:00:00  31.05.19 07:00:00  ok      60 60 0
IN  100004360  3426  30.05.19 16:00:00    05.11.2019                                           